References:
- This scraper is mainly adapted from the github repository: https://github.com/tw2734/Sephora-Fake-Reviews-Analysis/blob/master/Web_Scrapping_and_Data_Cleaning.ipynb
- https://github.com/Shirleyiscool/Scraping-Sephora/blob/master/scraper/scrape_reviews.py

In [4]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import sys
import time
from tqdm.notebook import tqdm_notebook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(f'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36')
path = '/Users/michellecheng/Desktop/SI630/chromedriver'

In [6]:
def get_product_url(url):
    #calculate total number of pages
    browser = webdriver.Chrome(executable_path = path ,options=options)
    browser.get(url)
    #cancel = browser.find_element_by_xpath('//button[@class="css-1kna575"]')

    #if cancel:
        #cancel.click()  
    #first_page = browser.find_elements_by_class_name("css-x544ax")
    try:
        next_pages = browser.find_elements_by_class_name("css-1f9ivf5")
        total_pages_count = next_pages[-1].text
        total_pages_count = int(total_pages_count)
    except:
        total_pages_count = 1

    #start scraping product names and links
    all_products = {}
    for i in range(1, 10):
        page_url = url + "?currentPage=" + str(i)
        browser.get(page_url)
        time.sleep(1.5)
        try:
            cancel = browser.find_element_by_xpath('//button[@class="css-1kna575"]')
            cancel.click()
            print('Clicked "Continue Shopping"')
        except:
            pass
        
        body = browser.find_element_by_tag_name("body")
        #body.send_keys(Keys.PAGE_DOWN)#.perform()
        #print(body)
        actions = ActionChains(browser)
        #body.send_keys(Keys.PAGE_DOWN)
        for _ in range(10):
            actions.send_keys(Keys.PAGE_DOWN).perform()
            time.sleep(0.1)
       # driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        #browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        source = browser.page_source

        results_page = BeautifulSoup(source, 'lxml')
        #print(results_page)
        product_grid = results_page.find('div', {'data-comp': "ProductGrid "})
        #print(product_grid)
        products_sections = product_grid.find_all('div', class_ = "css-1qe8tjm")
        #print(len(products_sections))
        #print(products_sections)
        for product in products_sections:
            #products = section.find_all('div', class_ = "css-1qe8tjm")
            #for product in products:
                if product.find('a'):
                    try:
                        product_name = product.find('span', class_ = "ProductTile-name css-h8cc3p eanm77i0").getText()
                        product_link = product.find('a').get('href')                    
                        #space = product_link.find(' grid')
                        #product_link = product_link[:space] + '%20%' + product_link[space+1:]                 
                        all_products[product_name] = product_link
                    except:
                        continue
        #print(all_products)
    browser.quit() 
    return all_products

#get_product_url('https://www.sephora.com/shop/skincare')

In [4]:
def get_user_info(username):
    browser = webdriver.Chrome(executable_path = path, options = options) # user-specific Chrome webdriver path
    browser.get('https://www.sephora.com/users/' + username)
    
    source = browser.page_source
    results_page = BeautifulSoup(source, 'lxml')
    
    try:
        info_content = results_page.find('div', {'class':'css-nu5x0t eanm77i0'})
        info_items = info_content.find_all('div', {'class':'css-f61fcm'})

        posts_made = int(info_items[0].get_text())
        reviews_made = int(info_items[3].get_text())
        
    except:
        pass
   
    return [reviews_made, posts_made]

In [11]:
def get_product_reviews(product_url):   

    browser = webdriver.Chrome(executable_path = path, options = options) # user-specific Chrome webdriver path
    browser.get(product_url)
    
    time.sleep(1.5)
    try:
        cancel = browser.find_element_by_xpath('//button[@class="css-1kna575"]')
        cancel.click()
        print('Clicked "Continue Shopping"')
    except:
        pass

    source = browser.page_source
    results_page = BeautifulSoup(source, 'lxml')
    
    
    i = 1
    all_reviews = []
    
    actions = ActionChains(browser) 
        
    for _ in range(10):
        time.sleep(0.1)
        actions.send_keys(Keys.PAGE_DOWN).perform()
            
    source = browser.page_source
    results_page = BeautifulSoup(source, 'lxml')
        
    #print(results_page.find('p', {'class': 'css-1bi3tmq eanm77i0'}).get_text()[5:])
    item_id = int(results_page.find('p', {'class': 'css-1bi3tmq eanm77i0'}).get_text()[5:]) 
    product = results_page.find('span', {'class': 'css-1pgnl76 e65zztl0'}).get_text()
    brand = results_page.find('a', {'class': 'css-1gyh3op e65zztl0'}).get_text()
    reviews = results_page.find_all('div', {'class': "css-13o7eu2 eanm77i0"}) 

    try:
        review_num = results_page.find('span', {'class': "css-nv7myq eanm77i0"}).get_text()
        review_num = review_num[:review_num.find(' Reviews')]

    except:
        review_num = ''

        
    while i < 61:
        
        if i * 6 > int(review_num):
            break
        
        actions = ActionChains(browser) 
        source = browser.page_source
        results_page = BeautifulSoup(source, 'lxml')
        reviews = results_page.find_all('div', {'class': "css-13o7eu2 eanm77i0"})
        
        for review in reviews:
            try:
                review_texts = review.find('div', {'class': "css-k7hahd eanm77i0"}).get_text() 
            except:
                review_texts = ''

            try:
                review_title = review.find('h3', {'class': "css-m9drnf eanm77i0"}).get_text() 
            except:
                review_title = ''

            try:
                user_name = review.find('a', {'data-at': "nickname"}).get_text()
                #reviews_made = get_user_info(user_name)[0]
                #posts_made = get_user_info(user_name)[1]

            except:
                user_name = ''
                reviews_made = ''
                posts_made = ''

            try:
                review_rating = review.find('span', {'class': "css-mu0xdx"}).get('aria-label')[0] 
            except:
                review_rating = ''

            try:
                review_votes = review.find_all('button', {'class': "css-36ie0l"})
                helpful_votes = review_votes[0].find('span').get_text()
                helpful_votes = helpful_votes[1:helpful_votes.find(')')] 
                not_helpful_votes = review_votes[1].find('span').get_text()
                not_helpful_votes = not_helpful_votes[1:helpful_votes.find(')')]

            except:
                helpful_votes = ''
                not_helpful_votes = ''

            try:
                verified_purchase = review.find('span', {'class':'css-gtt1cr eanm77i0'}).get_text()
                verified_purchase = 1

            except:
                verified_purchase = 0

            try:
                sephora_employee = review.find('span', {'class':'css-1clbni9'}).get_text()
                sephora_employee = 1

            except:
                sephora_employee = 0

            try:
                incentivized = review.find('span', {'class':'css-ezht25'}).get_text()
                incentivized = 1

            except:
                incentivized = 0


            all_reviews.append([item_id, product, brand, review_num, user_name, review_title, review_texts,
                                review_rating,helpful_votes,not_helpful_votes, verified_purchase, sephora_employee, incentivized])#, free_product, recommend, review_time, reviews_made, posts_made,])
        
        next_comments = browser.find_element_by_xpath('//button[@class="css-2anst8"]')
        desired_y = (next_comments.size['height'] / 2) + next_comments.location['y']
        window_h = browser.execute_script('return window.innerHeight')
        window_y = browser.execute_script('return window.pageYOffset')
        current_y = (window_h / 2) + window_y
        scroll_y_by = desired_y - current_y

        browser.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
        
        browser.find_element_by_xpath('//button[@class="css-2anst8"]').click()
        #actions.move_to_element(next_comments).click().perform
        next_comments = browser.find_element_by_xpath('//button[@class="css-2anst8"]')
        time.sleep(0.5)

        i += 1
        print(i)
        print(user_name)


    columns = ['item_id', 'product', 'brand', 'total_reviews', 'user_name','review_title', 'review_text', 
               'review_rating', 'helpful_votes', 'not_helpful_votes', 'verified_purchase',
               'sephora_employee','incentivized'] #,'free_product', 'recommend', 'review_time','reviews_made','posts_made',]

    df = pd.DataFrame(all_reviews, columns=columns) 
    
    return df

In [7]:
url_dict = get_product_url('https://www.sephora.com/shop/skincare')
len(list(url_dict.values()))

Clicked "Continue Shopping"


467

In [8]:
list_of_urls = list(url_dict.values())
list_of_urls

['https://www.sephora.com/product/creme-de-la-mer-moisturizing-cream-P416341?skuId=1932201&icid2=products grid:p416341:product',
 'https://www.sephora.com/product/augustinus-bader-the-cream-with-tfc8-face-moisturizer-P470507?skuId=2458875&icid2=products grid:p470507:product',
 'https://www.sephora.com/product/dr-barbara-sturm-hyaluronic-serum-P447212?skuId=2269892&icid2=products grid:p447212:product',
 'https://www.sephora.com/product/tatcha-the-silk-sunscreen-mineral-spf-50-P481169?skuId=2535870&icid2=products grid:p481169:product',
 'https://www.sephora.com/product/caudalie-premier-cru-anti-aging-serum-P480284?skuId=2534337&icid2=products grid:p480284:product',
 'https://www.sephora.com/product/guerlain-abeille-royale-youth-watery-anti-aging-oil-P474953?skuId=2492858&icid2=products grid:p474953:product',
 'https://www.sephora.com/product/peace-out-blemish-balm-cleanser-P469530?skuId=2445831&icid2=products grid:p469530:product',
 'https://www.sephora.com/product/facial-treatment-essen

In [261]:
df[(df['incentivized']==1) & (df['verified_purchase'] == 1)]#['review_text'].iloc[4]

,item_id,product,brand,total_reviews,user_name,review_title,review_text,review_rating,helpful_votes,not_helpful_votes,verified_purchase,sephora_employee,incentivized
1063,2181006,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,"2,514",StormieP,Best Moisturizer of All Time,I have extremely dry skin and have never loved...,5.0,4.0,1.0,1,0,1
1065,2181006,The Dewy Skin Cream Plumping & Hydrating Moist...,Tatcha,"2,514",aquaile,NaN,"I received this as my birthday item, and it's ...",5.0,2.0,0.0,1,0,1
1984,2549400,Watermelon Glow Niacinamide Sunscreen SPF 50,Glow Recipe,897,MorewoodJ,NaN,"I was so excited to try this product, but when...",2.0,5.0,0.0,1,0,1
2588,2180941,Superberry Hydrate + Glow Dream Mask with Vita...,Youth To The People,"2,163",Viljja,Great,Absolutely love this. I got is as a free sampl...,4.0,3.0,1.0,1,0,1
2952,1499482,Alpha Beta® Extra Strength Daily Peel,Dr. Dennis Gross Skincare,"7,250",RoxyKis,Worth the price,I heard about this product through a friend an...,5.0,9.0,0.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,2114486,Keep Young and Beautiful Anti-Aging Eye Cream,REN Clean Skincare,65,VaneSanchezK,Not that moisturizing,"Great for sensitive skin, not at all each or a...",3.0,6.0,2.0,1,0,1
3791,2398287,Supplement Case,SEPHORA COLLECTION,71,josiesdaughter,So very cute and compac!,This case is so cute and love the seperate com...,5.0,0.0,0.0,1,1,1
4054,2255271,Oil and Pore Control Mattifier Broad Spectrum ...,Murad,520,Carter0215,Can I give this SPF 100 Stars?,"I have combo-oily, dehydrated congested skin a...",5.0,34.0,1.0,1,1,1
4056,2255271,Oil and Pore Control Mattifier Broad Spectrum ...,Murad,520,Alyxo7,NaN,I first got this product as a sample at sephor...,1.0,7.0,5.0,1,0,1


In [98]:
df[(df['incentivized']==1) & (df['sephora_employee'] == 1)]['review_text'].iloc[96]

"I am a huge fan of this moisturizer! I got to a point where I wasn't really happy with my previous moisturizers either because I didn't receive enough hydration or didn't notice any results. I noticed immediate results with this product, giving me the hydration I need and being both lightweight but moisturizing. I also notice after using it for the past few weeks less dullness to my skin overall. If you're looking for a nice moisturizer and haven't found one you've really enjoyed, I highly recommend this one!"

In [262]:
df[df['incentivized'] == False]['review_text'].str.lower().str.contains('influenster').sum()

323

In [263]:
df[df['incentivized'] == False]['review_text'].str.lower().str.contains('voxbox').sum()

33

In [264]:
df[df['incentivized'] == False]['review_text'].str.lower().str.contains('sample').sum()

3244

In [165]:
df[df['verified_purchase'] == False]['review_text'].str.lower().str.contains('ordered').dropna()

0

In [187]:
df[(df['verified_purchase'] == False) & (df['incentivized'] == False)]['review_text'].dropna()[df[(df['verified_purchase'] == False) & (df['incentivized'] == False)]['review_text'].str.lower().str.contains('months').dropna()]#.iloc[2]

7        This cream may be a miracle for some people, b...
37       This product is like no other for dry skin. I ...
45       Got a 7mL sample size through a promo and I th...
51       Ugh. Sigh. I love the smell of this product. A...
53       I started using La Mer in 1990. Others noticed...
                               ...                        
13125    6 months after coming off birth control, I sta...
13146    I cannot day enough about this company and how...
13148    Would highly recommend doing a trial run & sav...
13154    I am a person who struggles with cystic acne m...
13185    Love this set. The quality of the bags the pro...
Name: review_text, Length: 1655, dtype: object

In [238]:
columns = ['item_id', 'product', 'brand', 'total_reviews', 'user_name','review_title', 'review_text', 'review_rating',
           'helpful_votes', 'not_helpful_votes', 'verified_purchase','sephora_employee','incentivized']
data15 = pd.DataFrame(columns = columns)
x = 1
for url in tqdm_notebook(list_of_urls):
    try:
        data15 = pd.concat([data15, get_product_reviews(url)])
    except:
        continue
    print(f'------- URL {x} Done --------')
    time.sleep(1)
    x += 1

  0%|          | 0/535 [00:00<?, ?it/s]

2
Pem410
3
PalomitaAA
4
sephormore
5
duchessmatisse
6
Ljuba
7
chicabonita324
8
anxious
9
LaurenCL
10
abett
11
LouLouR
12
cruzcatiii
13
zingaz
14
JSFlyer
15
ciccastars
16
hoppie
17
adelineeee
18
nolachris
19
redjoyce
20
ardena32
21
PurdueGal
22
athena321
23
sullj
24
crzysvety
25
liliquoi
26
lulakilla
27
susiecue
28
kensingtonbeauty
29
nutmegmace
30
bee814
31
brinny
------- URL 1 Done --------
2
boomshakalakaa
3
NickKnack
------- URL 2 Done --------
2
castmelanie
3
nmribs
------- URL 3 Done --------
2
Miicchhelle
3
mimi0506
4
kimber0917
5
AveryJBrio
6
brooklynn0404
7
SuzieQ73
8
missmarybears
9
BWalter
10
Megsasaurus
11
kokeshiBlue
12
Ohgeezi
------- URL 4 Done --------
2
Jessiekaur1
3
grannybanka
4
danim
5
CuriousTanya
6
alirhea22
7
mythica79
8
lovesumsephora
9
Chicspace
10
linzerC
11
lindy0116
12
barbs33
13
SADOLORD
14

15
debb0202
------- URL 5 Done --------
2
jrye
3
lavenderrose117
4
halosandcrowns
5
arielkez
6
OStahl
7
Taylorv1
8
aline
9
beachbum510
10
bhgirl
11
cris9223
12
Steffy21


5
jjwiththements
6
JB593
7
MaddieB105
8
pablita
9
Gizmo2019
10
basketballer98
11
ouaimilk
12
alanabiggie
13
katiebarchie
14
kimalone84
15
breezy9892
16
jsendslove
17
alybo
18
grossgurl
19
MrsCharles
20
AwkwardAshlea
21
kaitlyml
22
Aishahg
23
BrandiC728
24
apearlr
25
ffxgirl1
26
AddieBanana
27
arielmariex
28
naomibadillo
29
landseasky
30
mayritaxo
31
KatJ27
------- URL 59 Done --------
2
jennfrommd
3
tkutch
4
Farzee
5
ChanelLover15
6
angelchick
7
losangelite
8
Jg2023
9
brittherrmann
10
Moon919
11
AshleeSwindle
12
KAbb52
13
kateelizabeth27
14
RSGrissom
15
haloga
16
ej1230
17
sg1988
18
cerah2021
19
aperke
20
Bettskrum
21
aiex
22
lorraineaj
23
xokeebuuxo
24
winnwng
25
crissie225
26
jjsfunny
27
BackwoodsBarbi
28
balletlove
29
bettyiglesi
30
julia176
31
hannahorellana
------- URL 60 Done --------
2
itsarii
3
salazarsophia
4
ashhh666
5
kittypippin
6
MNikkila
7
Ruelette92
8
Madiilip
9
anyaste
10
Mariaamsh
11
JJM86
12
AbbyGooe
13
moncheriesarah
14
AmandaMaria92
15
mothprincess
16
mysito
17
ella

20
jengbrecht
21
Jp1998
22
popsugarkiss
23
DaniBellXo
24
beatylover4er
25
emwitten216
26
neecoles
27
OhHeyMikki
28
teenmist
29
Kekkz
30
Maresli
31
AngsB
------- URL 107 Done --------
2
jocchi7
3
kita08
4
beautyqueensd
5
CharlotteGold
6
MasyATomlinson
7
theurbanfox
8
toribaby
9
Betka17
10
prettygeek18
11
MiaKast
12
fefe05
13
jhu545
14
RachieBabie
15
Annafsnyder
16
rubimala
17
ClareMicMac
18
PamA89
19
KathrynRob
20
Sarahy95
21
AnaHoneyBear
22
Anneopinion
23
Loralie379
24
Lizxv9
25
kayrosefed
26
arisha12
27
jescastro
28
jrl123
29
amirap
30
akayla92
31
kcnyc14
------- URL 108 Done --------
2
PrincessNino
3
Brizicane
4
cmacfli
5
Sun21
6
Susan10013
7
liljesss
8
gonzaga9205
9
simplycharlotte
10
mrseth
11
JennyD100
12
LlamaLover
13
MissBS
14
Simone24
15
AliB123
16
melventuri
17
jstme1986
------- URL 109 Done --------
2
jasrice
3
lydzma
4
GeekyNeko
5
Riley710
6
ImMai
7
mdrmpls
8
emce13
9
emrg
10
leerwren
11
guerr
------- URL 110 Done --------
2
shabrin07
3
julianneT123
4
dilainaSHEA
5
hh11
6
Ab

22
buffyfanify
23
LuminousAK
24
katelardner
25
AlexSo
26
Mariaramirezm
27
HAGill
28
jennifery08
29
KUJIRAnoOUJI
30
Msjmela
31
swiderski72
------- URL 151 Done --------
2
TatyanaLockwood
3
SaronGeb
4
sess42
5
MissOliviaF
6
MeliB7
7
blondebamabelle
8
Latrice345
9
jennieivanka
10
AshleyAGlass
11
ellieP414
12
ajkon9
13
SuzieAdd
14
toryj9066
15
mcarthurm
16
hamptonpilates
17
joannehaznyc
18
Dorinm
19
FeliciaBelle
20
Bre285
21
zoha2801
22
anibananie
23
Sealj
24
simasimaa
25
sugarduckface
26
Angiejolie02
27
emchampagne
28
SlimMiah
29
AutumnBee
30
BeautyyJunkie91
31
lil4tentechie
------- URL 152 Done --------
2
Hlozo
3
SaraVilhelmina
4
meerkat5
5
lidianams
6
laurgal222
7
lcamp12
8
GamesAndGlamour
9
Camila
10
InbalMiller
11
realityxbites
------- URL 153 Done --------
2
RubiconDana
3
sashajane
4
Carina7
5
junoraven
6
lucee623
7
fitmommy91
8
dharmanla
9
ppzhang
10
Luv2Golf2
11
Lonea
12
Laurenf1862
13
mmgr
14
BeautySnobb
15
BlueSmurf
16
1makeuplover1
17
Nadine21
18
nana808
19
girlonsunday
20
LeeLe

------- URL 202 Done --------
2
mehitabel98
3
christyjoyx
4
Jenjen128
5
savannah1003
6
veejuu45
7
BeeLab
8
ShardaeMarie
9
ChezAyesha
10
elizalbany
11
bkby
12
dietcokegrrl
------- URL 203 Done --------
2
haileybailey93
3
nikiiiiiii
------- URL 204 Done --------
2
NeniOla
3
sparky81
4
deardeers
5
Megg100
6
yunglolita
7
meowmamasita
8
ftworthbeauty
9
Bink1026
10
nathali23
11
fyrefly1
12
sydneyynicholee
13
JesTheRed
14
Dthom12
15
Deleahna
16
Tetum1
17
Thegunpowder
18
Kylee97
19
Keyks
20
Trnhpooh
21
cassiewilde
22
freshhprincess
23
melrel2005
24
gingerHR
------- URL 205 Done --------
2
suzukipeach
3
rachelshikhman
4
Verabreu
5
serenah89
6
Pokeey
7
fatimaab91
8
thefav0rite
9
Raveygurl
10
Elicia146
11
Littlepurple
12
Rach7119
13
darthb
14
samomal
15
bamur
16
nargz503
17
rachmae411
18
Tine112
19
c21c3
20
Moni09
21
rachieer
22
countrygal921
23
ValGon
24
PGKey
25
EmAbraham
26
caslatta
27
ladykrysta
28
SomeNameHere
29
madfedss
30
megmeg123
31
lindahirv
------- URL 206 Done --------
2
MrDon
------

2
kwx913
3
augustina
------- URL 245 Done --------
2
HEATHMP
3
VaniaCarolina
------- URL 246 Done --------
2
makeupfinder
3
nath001
4

5
teejay2000
6
beautyjunkie1011
7
SivanBlogs
8
MKEJordan
9
JuliaLGulia
10
QTBeauty79
11
gina410
12
Ellenvmt
13
katkatmc
14
Mercedes33
15
Shanaes
16
KiraKira72
17
KeekSkin12
18
nankat74
19
JDHtheGR8T
20
DesireeEloiza
21
Billeh
22
PedroBrazil
23
withloveShanel
24
jmwilson19
25
Benefield
26
Canarybirdkarla
27
miamomia
28
CC84
29
juliaccash1
30
Smallwonder2003
31
Michelle204
------- URL 247 Done --------
2
ashleysawatsky
3
bhbbzza18
4
jenniferle98
5
Pri23Chaya
6
belreyes18
7
Bruinette16
8
btyismylfe
9
aure
10
aure
11
juliaelaine
12
bdurant
13
xhiba
14
eaazye
15
ChaunieNiks
16
Sujey
17
amanda2010
18
tamiig
19
ladywest
20
dilv
21
Morguinha
22
bekahsometimes
23
Slaybec
24
1aura
25
yurimon
26
db01
27
nyc2atl2dc
28
lauralyn1
29
jeankn
30
mea09
31
hannahfostr
------- URL 248 Done --------
2
Aphrodite75
3
lc022
4
emsroeder
------- URL 249 Done --------
2
Kserasera

4
AnaL95
5
emmaamargarett
6
fukyou
7
slcrx
8
pmotz
9
MaliaYYZ
10
mirmski
11
kycharlie
12
Eleesandra
13
jmegan
14
WorthExploring
15
Pigeonlady
16
danseymour
17
batool99
18
ke3515
19
Atoussa
20
Suri89
21
nancy008
22
QiQichi
23
prettyluv
24
makeupaddictrn
25
mollymo
26
iremom
27
dreamyone
28
maryamh
29
smyr
30
krinoline
31
inni
------- URL 279 Done --------
2
mayte07
3
tara12
------- URL 280 Done --------
2
teampp1223
3
sparkle24u
4
skylarquinn
5
maryga
------- URL 281 Done --------
2
dementedbarbie
3
teeeeeebone
4
LeicaLashes
5
DJ18
6
yezebel
7
TO2018
8
DoraKuc
9
sirena
10
Alohaloha
11
broese
12
Caicai233
13
Japanesenoodle
14
SiteNo8
15
BruinGal03
16
Pixiee
17
mb85
18
snapster
19
amirag
20
amandamom
21
xstefaniex
22
alabasterglow
23
ambar09
24
hrehlers
25
craftzombie
26
lcgs
27
ctgal1
28
skincarefan1
29
dvs1121
30
sheriseas
31
vitamin
------- URL 282 Done --------
2
baerro
3
Deboratory
------- URL 283 Done --------
2
turtlewexler
3
jkaler
4
nehaxoxo
5
ocsheena85
6
GemBilbo
7
cstass
8
jen

21
becca30
22
Carola1
23
Alita07
24
Christal71
25
flor123
26
Su42
27
Azanique
28
dville79
29
angienovi
30
AmberGriff
31
Springn89
------- URL 324 Done --------
2
DLTH
3
MsLadyJay
4
ChrisSnow
5
sbean132
6
OGdaelynn
7
morganelliott
8
MadHip
9
antheawalker
10
Elivior
------- URL 325 Done --------
2
JKrych85
3
KMMT
4
lisa304
5
SopZha
6
mommykym
7
Styzzie
8
Annieholds
9
Zinia106
10
emilymskaggs
11
claudiacostam
12
KatieJ11
13
TeaganC
14
sunshine300
15
MimiLe
16
sarahmtroxell
17
Giannamarrie
18
IrisBand
19
andreedree
20
spicyskydiver
21
amanduhcawl
22
calliebircher
23
kirstykirst
24
Emily00008
25
stephanini4555
26
mayams7
27
kglorioso
28
Mandy19191
29
adorestarla
30
VAdki
31
mpcqw
------- URL 326 Done --------
2
Rayybabyyy
3
sininasac
4
Nuria03
5
Steph8910
6
thisiskim
7
kassonja
8
AleMaria8
9
baharjoon
10
anakrist
11
prettylady80
12
KaileyKeltner
13
evvvvvvva
14
claytonasaurus
15
giraffesnax
16
Kendra1486
17
katiek533
------- URL 327 Done --------
2
patitaaa
3
Mkang89
4
YennyBoo
5
danny07305

In [239]:
#data15.to_csv('sephora_data15.csv')

In [26]:
df1 = pd.read_csv('sephora_data.csv')
df2 = pd.read_csv('sephora_data2.csv')
df3 = pd.read_csv('sephora_data3.csv')
df4 = pd.read_csv('sephora_data4.csv')
df5 = pd.read_csv('sephora_data5.csv')
df6 = pd.read_csv('sephora_data6.csv')
df7 = pd.read_csv('sephora_data7.csv')
df8 = pd.read_csv('sephora_data8.csv')
df9 = pd.read_csv('sephora_data9.csv')
df10 = pd.read_csv('sephora_data10.csv')
df11 = pd.read_csv('sephora_data11.csv')
df12 = pd.read_csv('sephora_data12.csv')
df13 = pd.read_csv('sephora_data13.csv')
df14 = pd.read_csv('sephora_data14.csv')
df15 = pd.read_csv('sephora_data15.csv')
df16 = pd.read_csv('sephora_data16.csv')
df_all = pd.concat([df1,df2,df3,df4, df5,df6,df7,df8, df9,df10,df11, df12, df13,df14,df15, df16])
df_all

,Unnamed: 0,item_id,product,brand,total_reviews,user_name,review_title,review_text,review_rating,helpful_votes,not_helpful_votes,verified_purchase,sephora_employee,incentivized
0,0,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",flDeb,Too thick and sticky.,More like a sticky paste than cream. I also do...,1.0,0.0,1.0,1,0,0
1,1,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",KitKatChi2018,Heaven for glowy normal to combo skin,I was NOT expecting to love this moisturizer a...,4.0,2.0,0.0,0,0,0
2,2,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",Scorpiobabe93,Nice but ridiculously overhyped.,It’s moisturizing and definitely feels very lu...,3.0,13.0,5.0,0,0,0
3,3,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",purrrsuasive1,NaN,I have been using this for years 1 ounce last ...,5.0,12.0,13.0,0,0,0
4,4,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",EmilijaLVS,Not worth the hype,I really don’t get the hype. I got a mini size...,2.0,26.0,8.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,7,2192938,Pink Perfection Blackhead Extractor,TWEEZERMAN,16,nita1228,Waste of money,Waste of money. Can not grip onto anything fin...,1.0,45.0,9.0,0,0,0
5148,8,2192938,Pink Perfection Blackhead Extractor,TWEEZERMAN,16,EDM13,NaN,NaN,1.0,8.0,35.0,0,0,0
5149,9,2192938,Pink Perfection Blackhead Extractor,TWEEZERMAN,16,radi0head,NaN,This just doesn’t work at all. Awkward design ...,1.0,19.0,5.0,0,0,0
5150,10,2192938,Pink Perfection Blackhead Extractor,TWEEZERMAN,16,lmaster,NaN,I actually had a wonderful experience with thi...,5.0,128.0,22.0,0,0,0


In [25]:
df_all.to_csv('sephora_review_data.csv')

In [27]:
df = df_all.drop('Unnamed: 0', axis = 1).drop_duplicates()
df.shape

(138154, 13)

In [259]:
df['incentivized'].value_counts()

0    95068
1    43086
Name: incentivized, dtype: int64

In [260]:
df['verified_purchase'].value_counts()

0    124310
1     13844
Name: verified_purchase, dtype: int64

In [267]:
df[df['incentivized'] == 1]['review_rating'].mean()

4.469618901731421

In [268]:
df[df['incentivized'] == 0]['review_rating'].mean()

4.002263348492504

In [21]:
print(df[(df['incentivized'] == 0)&(df['verified_purchase'] == 0)]['review_rating'].mean())
print(df[(df['incentivized'] == 0)&(df['verified_purchase'] == 1)]['review_rating'].mean())
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 1)]['review_rating'].mean())
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 0)]['review_rating'].mean())
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 0)&(df['sephora_employee'] == 1)]['review_rating'].mean())

4.033009823881718
4.071656427221172
4.618705035971223
4.507920402229222
4.5929824561403505


In [20]:
print(df[(df['incentivized'] == 0)&(df['verified_purchase'] == 0)].shape[0])
print(df[(df['incentivized'] == 0)&(df['verified_purchase'] == 1)].shape[0])
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 1)].shape[0])
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 0)].shape[0])
print(df[(df['incentivized'] == 1)&(df['verified_purchase'] == 0)&(df['sephora_employee'] == 1)].shape[0])

101261
16928
417
66032
570


In [269]:
df[df['verified_purchase'] == 1]['review_rating'].mean()

4.051863623230281

In [270]:
df[df['verified_purchase'] == 0]['review_rating'].mean()

4.158821256660818

In [271]:
df[df['verified_purchase'] == 1]['helpful_votes'].mean()

8.103655013002022

In [272]:
df[df['verified_purchase'] == 0]['helpful_votes'].mean()

4.255888082167522

In [273]:
df[df['incentivized'] == 1]['helpful_votes'].mean()

1.2734763032075385

In [274]:
df[df['incentivized'] == 0]['helpful_votes'].mean()

6.169403739262254

In [275]:
df[df['incentivized'] == 1]['not_helpful_votes'].mean()

0.5708814928282969

In [276]:
df[df['incentivized'] == 0]['not_helpful_votes'].mean()

1.710670372241873

In [12]:
columns = ['item_id', 'product', 'brand', 'total_reviews', 'user_name','review_title', 'review_text', 'review_rating',
           'helpful_votes', 'not_helpful_votes', 'verified_purchase','sephora_employee','incentivized']
data17 = pd.DataFrame(columns = columns)
x = 1
for url in tqdm_notebook(list_of_urls):
    try:
        data17 = pd.concat([data17, get_product_reviews(url)])
    except:
        continue
    print(f'------- URL {x} Done --------')
    time.sleep(1)
    x += 1

2
kyliesparkle
3
SaraLouWho
4
Aloraaa
5
clairewurzbach
6
sprice77
7
KLuca
8
DaisySixSpeed
9
jackiebaird
10
IrenShol
11
liswoods
12
atypicalme
13
rvague
14
jadedbeauty
15
elanmc16
16
XXX123
17
Sandyphoenixx
18
laccc
19
ShaeG
20
krae1239
21
eagey1193
22
ledanger
23
nikkiflnew
24
BeautyFanatic76
25
momo212121
26
bluesyblonde
27
achvzz
28
annej1669
29
Ent52711
30
marshmeg
31
clairedabear
32
RebPal
33
BBeis
34
jasmine57
35
lexmm
36
GwennieGott
37
avelasquez
38
chasingthedream
39
theglambelle
40
tibone
41
Rosexix
42
kbjb164
43
LTor88
44
courtyg13
45
mellyw
46
WinkyZinky
47
Kerbom
48
badaing11
49
sedonah11
50
Luda22
51
gabichkagirl
52
Toto1992
53
jamesthegay
54
Fancying
55
freshflesh
56
savvyshopper2
57
JustJessin
58
shitimintoo
59
BeatriceV
60
petitesrule
61
AgAudi
------- URL 1 Done --------
2
voguekitty
3
Weffodee
4
Gr00mAddict
5
jj625
6
shreya1809
7
carline
8
sundial
9
FancyNa
10
goodbyemoney
11
jjlee
12
Bellerina15
13
SelenaFaith
14
HEB369
15
anamishu
16
komwal349839292
17
Mimi0787
18
be

46
mck2000
47
akty
48
nt61
49
kitty425
50
Emilydomenica
51
TinaLOL888
52
Sauccey
53
CristyEvans
54
JoseBon
55
maegenmakeup
56
Lenannedez
57
janellep
58
Jessica18911
59
feebee630
60
Saralynnsmiled
61
KoiNami
------- URL 15 Done --------
2
ashleyliz6
3
angel22eyes
4
Yingho
5
ahess0817
6
erinmeaney
7
Ollypops19
8
NatashaR423
9
SMRW
10
Aiyel
11
ShayNOLA
12
itsnealnotmeal
13
birdysparadise
14
AnnetteCielo
15
Violetstorm78
16
jini977
17
cariocagold
18
MyTopThingsss
19
sunalbina
20
SophiaCranmer
21
shay5411
22
nikki2dewwan
23
laurentttt
24
berniece25
25
berniece25
26
berniece25
27
wingitpretty
28
Mominaaab
29
Mominaaab
30
pbg37
31
StephTor
32
StephTor
33
Mbodle
34
gizmodie26
35
Breanmon
36
weverynametaken
37
weverynametaken
38
suhanathan
39
deeoonn
40
abzstylz
41
DayanaZ
42
Ttml
43
cnorden
44
wendykc97
45
ambertiffanny
46
HanaRoxanne
47
Aimeee93
48
testrite
49
CharRawrz
50
CreoleNichee
51
MannyMichael
52
valtrovato
53
sayeri
54
Qfglow
55
squidney246
56
Yaelfromla
57
marguerite01
58
Jaxfarinas

4
LanaVol
5
snowde88
6
vee831
7
rae1278
8
tmcook22002
9
Sweetpie88
10

11
amyxaz
12
ChitraH
13
Gunslinger45611
14
SunzJagz
------- URL 28 Done --------
2
amelar
3
kitkat919
------- URL 29 Done --------
2
heer28
3
Aneza
4
aadanci
5
MyeishaG
6
ohenryjackson
7
MaddieRichter
8
McKennaMunden
9
MaddiCochran
10
ritamaksimchuk
11
Taylorallen10
12
KristinaLachaga
13
MawiesTwisted
14
Mvalle12
15
simplysethh
16
Brittanyhope10
17
ammerm12
18
AlisonRS
19
lindseyam011
20
madisonborgel
21
Arivdna
22
hellogabrielle
23
rayaneee
24
mrebalkin
25
SSO14
26
luara12222
27
Ashlilybrooke
------- URL 30 Done --------
2
JayDee89
3
frankie24
4
jnk013
5
jdavis5787
6
sharklovingmom
7
winnielee
8
KellieKells
9
RamaG22
10
trangilicious
11
enska
------- URL 31 Done --------
------- URL 32 Done --------
2
rosegardens
3
AAAHHHH
4
muchglens
5
gigi1987
6
kalikiepua
------- URL 33 Done --------
2

3
phiavitti
4
erikanls
5
britb2
6
teooo
7
tmille
8
Blond3
9
MechyV
10
angoula
11
MuscleBabe
12
RaychelRondo
13
stalu
14
MsWgz
1

21
hannahwho
22
Angelikoula
23
lindZG207
24
snaidero
25
Annie1981
26
MLou18
27
maryydixonn
28
moniqueh11
29
Estuhz
30
Gooseisthebest
31
SarahConnor
32
KKellyImages
33
SaraJW7
34
Mairaaa
35
nanzobananzo
36
mallard1989
37
Rach616
38
JishirllJaneesa
39
jeelgo
40
nathacia
41
DokiDoki232
42
K8Cally
43
Kerrrrrrrrrry
44
finlou
45
Neen5515
46
msafrikanachic
47
sjcharest
48
NinaKara
49
elodzsun91
50
startrekkie
51
heybianca
52
Soniba2
53
shee18
54
shee18
55
comradezorn
56
ao130
57
MKbeyourself
58
kenziecai
59
Kevvvykevv
60
bbbbbmariep
61
KaiKai0369
------- URL 48 Done --------
2
KPouteaux
3
Laurenbobo
4
jaclynmg
5
paulinea
6
ElleAW
7
ChrissyB007
8
skincarexoxo
9
DreaBB
10
cosmaddiction
11
customer12321
12
xnlpjx
13
NikkiRunning
14
Jamrl16
15
Snooz521
16
StarMagnolias
17
SugarPlum01
18
MichelleJB
19
mosquito7
20
Maye2412
21
susitah88
22
mattexglow
23
LeAndra143
24
1234beaut
25
BFloNiki
26
txrn2019
27
tata1984
28
rawbeebee
29
saba2007
30
clr74
31
sarahm3758
32
JennnONE
33
kaysizzle
34
Budziak
35


51
nasnousa
52
missashley14
53
matt0122
54
myabrianne
55
lindsay52
56
NicoleS11
57
Bumblebonn
58
Rishelly
59
cromags
60
PamDarnell
61
FindAllyLove
------- URL 63 Done --------
2
jaclynmg
3
kfn321
4
dulcedagger
5
becomejaded
6
Myrdae
7
Stephanieb1
8
ajdth
9
cptkns
10
Savaney
11
TessMarCo
12
BJO4321
13
Ashleighrenee42
14
CarolineB02
15
ericaaaab
16
ArielaMassotti
17
kendallgay
18
awree
19
emulationn
20
nessa223
21
Maninylah
22
joeythelamb
23
holly927
24
Adobea
25
Beautyb77
26
isa23law03rusi1
27
thewhiteplate
28
Evilgirl
29
emsunu
30
nytginger
31
RebeccaP43
32
Sarahlee1995
33
sephraanthony
34
Sakeller4
35
addictskincare
36
Giovannacj
37
leihdii
38
ladybaltazar
39
loveituseit
40
erickita
41
Shahrzad68
42
Pampam14
43
ChristellaA
44
msbecca08
45
biancardi
46
RuzikM
47
MeganMayday
48
itsaislingduh
49
ymkmb
50
Mirilulu
51
NancyAde
52
tori191
53
quay
54
MirandaMarie89
55
vintageflats
56
alwaysally
57
mce1994
58
nickikayphoto
59
meimeimeimei
60
meimeimeimei
61
joicie
------- URL 64 Done --------

...

14
Alegre19
15
lilyzman13
16
Anna786
17
jmaciel233
18
SephoraLesley
19
NehaSG
20
emlulo
21
Sharay4
22
MonetAllyse
23
ynippie702
24
CSsung
25
Aly2Boo
26
RandeeJ9
27
Priceless34
28
crissybug
29
britnbar
30
AMRLove
31
Baljmaa
32
StarDust13
33
Tinker90
34
Kcorcs12
35
rcdeman
36
kimberlyyyxo
37
LiLMichelle26
38
teapear
39
Laugh42
40
cinthiasalinas
41
laurapsd
42
surveygirl
43
drewq92
44
jesuiskiwii
45
TabiThoughts
46
mommy2424
47
koulandgo
48
bharmon1991
49
Jadephoenixmama
50
RadhaDyadat18
51
Briaaax
52
jmcortez15
53
wsaa
54
Jamieleehop4208
55
SaritaSesha
56
kikameza619
57
jadestones777
58
anylit
59
LouisaCao
60
fragyle
61
zoey3357
------- URL 169 Done --------
2
sydddxo
3
17rocky
4
irisy
5
perfetc42
6
Kerfluffwin
7
Hephzibah1
8
chocomochi3
9
RoxyBee
10
Anooshka
11
kristtiee
12
Mayra1214
13
AClovesT
------- URL 170 Done --------
2
Anita5689
3
DVANDEN123158
4
amycanZ
5
echarles92
6
jwelk08
7
SueL90
8
cutiepie1701
9
bchar
10
VidSet
11
savingmoneyhun
12
Shaniki60
13
Shuyin987
14
Sakurameow
15


15
AndreaJunelle
16

17
rainynikki
18
Dm30
19
monalovebeauty
20
Ameliap109
21
LouiseVie
22
lanaguide
23
MissMomJeans
24
alyjennae
25
JKSJ32
26
MeghanMDee
27
babibek
28
SavvyyLynn
29
Sajida007
30
SpecialK1215
31
SpecialK1215
------- URL 194 Done --------
------- URL 195 Done --------
2
cmariehb
3
DeliciousDiva
4
marieski
5
akchick24
6
search4youth
7
SeaJules
8
staciemclendon
9
ramsk1
10
skinluv428
11
Redranda
12
707AGA
13
kimleot
14
Arosiegal
15
RosyPasadena
16
CaliKC
------- URL 196 Done --------
2
AnnetteCielo
3
venussp
4
kaseyschempf
5
masvale
6
AlekseyA
7
helenkellerss
8
Leksey
9
mynamecantbefou
10
lindseyvang
11
dilainaSHEA
12
jezskin32
13
becca123490
14
Leslieeey
15
RachelMarie90
16
Mai0620
17
BayleeMcNew
18
jeje20
19
ItzCrissy7
20
wowokiko
21
bonemoavk
22
Lienngo918
23
Jiawen66
24
MishMish11
25
lilyyu03
26
bittubunu
------- URL 197 Done --------
2
kritter111
3
cai90
4
tamaramcgann44
5
chlobau
6
luckbethislady
7
Veronique04
8
AnnieCake
9
heyitsgina
10
TXbeauty99
11
DaisyGlam
12
lo

18
Femina19
19
Kellteich
20
Sandysshore
21
EmilySusan
22
Emmiline89
23
AnaL95
24
Martinezrubby
25
hamdabby
26
daniiiikins
27
jnrogers96
28
LisaAllison
29
dawnjean
30
grasshaps
31
pnchvncr
32
ChairwomanMeow
33
vanessaa7
34
Elaina1
35
ZoiM
36
justjackie824
37
ElaineKelsey
38
TiffaniRains
39
Devin3
40
SpringLilys
41
PerfSkinSeeker
42
gypsc
43
JCL823
44
MaddyMorency
45
drnic
46
noem7
47
AllieHerde
48
tracey13
49
Quinn5030
50
ShardaeMarie
51
kellymariefitz
52
Kaedynwilliams
53
ivannapatricia
54
mashaw
55
Roseycat
56
Alina182
57
Skycanfly
58
LilBil
59
MmmmSashimi
60
ahhnai
61
lovemakeup12345
------- URL 222 Done --------
2
B0OSKI
3
Boopakitty
4
Hope511
5
TFolsom
6
BanCutie
7
jaimepg
8
hannah9130
9
emiphiphi
10
CourtIrwin
11
TJHNW
12
nandeeta
13
tracylilac99
14
hayleysparkles
15
littlebee88
16
CTorosyan
17
dianaericaa
18
itsjaleejane
19
joeythelamb
20
DJedinak
21
bvddiebri
22
amuscella
23
Amoonduh
24
kkitty07
25
LindseyRRR
26
komwal349839292
27
corgo24
28
amyea
29
maddiedgymnast
30
Willow14
3

12
Strangempiee
13
beea6
14
mireyanotcarey
15
jessab8
16
fnsopel
17
Nmjmxo
18
Ahurd746
19
MrsBP
20
JennCantwell
21
laraib12
22
Jooodeeee
23
traceyjoe
24
Lgwood
25
sandyinsider
26
lindsmarie1987
27
Sparkle2323
28
gchad4
29
TheCoffeeista
30
whitneyinthesky
31
Larissa2021
32
Georgiaweb
33
sydneymil1
34
Chantalbreland
35
tiffrc77
36
18krissa
37
hunnyybeauty
38
thealderbeauty
39
Jeorge
40
JeanTee
41
Ninjachana
42
alprxncess
43
AmberAlonzo
44
Ellejimm
45
elhabanana
46
Catmom85
47
Briannasmith024
48
toniking
49
TartanSpade
50
hwong
51
howdyitsmeg
52
nclark2
53
iqrab
54
fitzyx0
55
styledthought
56
karaea
57
kinsiejo
58
amberangeline
59
cheslea
60
EngU
61
navykaur
------- URL 243 Done --------
2
jolima
3
Jessice89
4
jackiepera
5
erinelsie
6
SusanaGali
7
Orchidfox
8
hiBye1969
9
meganjd
10
karleighwebb
11
LivV13
12
LolaAnj
13
Jes528
14
iluuuuvpink
15
JoBeautyAddct
16
sarahmin
17
ridd
18
kiki2065
19
KMartini2
20
SashaM5
21
MegSuee
22
Hempat
23
deevabu
24
Kherrons
25
vs1211
26
CountessB
27
Briantwi

9
leahonthelake
10
S3lentz
11
NurseEdana
12
ladykraska
13
jmich25
14
jakrantz
15
MrsLaurenAsh
16
Jorane23
17
beautyaddict93
18
RobynPaigeH
19
lexib1316
20
neckupmakeup
21
steph202039
22
polizzi
23
KamiiBee
24
Babyleece
25
shortysreviews
26
jshsidnxhdjd
27
beautyholic90
28
Laurentastic
29
izzyFlores1
30
MonsterDolll
31
aqeela
32
kingi
33
MissyMonster
34
astaire
35
freshvibes
36
Taniipartner
37
LadyMariposa
38
KRoss820
39
JoJoMado
40
jayr777
41
tamarajoy
42
kcdiaz
43
JessssTheBest
44
killexia
45
exjm
46
agrismer
47
MsShekaBoo
48
Baruss
49
Melz27
50
Rhieye
51
Feltch
52
btbbs4
53
nordicbunny
54
JAZZMINS
55
Emmadowling55
56
Smrtlizz
57
Las728
58
odelaooo
59
arischikiss
60
AlexandraFM
61
BethanyDee
------- URL 261 Done --------
2
Gelsominaa
3
BritneySheehan
4
jak82
5
meganlbbh
6
wuq5277
7
laurendeg
8
deliriumtremens
9
caralots
10
Crystallane
11
pwet
12
edjona10
13
JealousGull
14
hmpocketglam
15
NandaLeite
16
sunglowerjess
17
babibek
18
HallieAllie
19
AZ1005
20
Sactown1
21
Mollyf28
22
letstac

In [13]:
data17.to_csv('sephora_data17.csv')

In [14]:
data17

,item_id,product,brand,total_reviews,user_name,review_title,review_text,review_rating,helpful_votes,not_helpful_votes,verified_purchase,sephora_employee,incentivized
0,2269892,Hyaluronic Serum,Dr. Barbara Sturm,378,nana5247,Not worth the $$$,Very expensive and there are similar products ...,2,1,0,0,0,0
1,2269892,Hyaluronic Serum,Dr. Barbara Sturm,378,ElizabethM27,Just buy The Ordinary + 3 weeks of groceries,"I received a sample of this, knew it would be ...",2,10,2,0,0,1
2,2269892,Hyaluronic Serum,Dr. Barbara Sturm,378,sunflower33,,I have no idea how people compare this serum t...,5,10,3,0,0,0
3,2269892,Hyaluronic Serum,Dr. Barbara Sturm,378,K9Agility,Feel cheated,"I love the product but I, like several others,...",3,12,1,0,0,0
4,2269892,Hyaluronic Serum,Dr. Barbara Sturm,378,dirtyunicorn,Shocked,I am shocked that I just spent over $100 on so...,2,33,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,CrazyxoLizzy,VERY HYDRATING,My face was screaming for hydration and the Th...,4,0,0,0,0,0
356,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,JBracamontes,just WOWWWW,I have been using this In the mornings and nig...,5,0,0,0,0,1
357,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,Jayaaah,AUH-MAY-ZINGGGG,"I have combination skin, an oily t zone and su...",5,0,0,0,0,0
358,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,saarah14,All Aboard the Hydration Train!,I have oily skin and I was surprised how amazi...,5,0,0,0,0,1


In [28]:
df1 = pd.read_csv('sephora_data.csv')
df2 = pd.read_csv('sephora_data2.csv')
df3 = pd.read_csv('sephora_data3.csv')
df4 = pd.read_csv('sephora_data4.csv')
df5 = pd.read_csv('sephora_data5.csv')
df6 = pd.read_csv('sephora_data6.csv')
df7 = pd.read_csv('sephora_data7.csv')
df8 = pd.read_csv('sephora_data8.csv')
df9 = pd.read_csv('sephora_data9.csv')
df10 = pd.read_csv('sephora_data10.csv')
df11 = pd.read_csv('sephora_data11.csv')
df12 = pd.read_csv('sephora_data12.csv')
df13 = pd.read_csv('sephora_data13.csv')
df14 = pd.read_csv('sephora_data14.csv')
df15 = pd.read_csv('sephora_data15.csv')
df16 = pd.read_csv('sephora_data16.csv')
df17 = pd.read_csv('sephora_data17.csv')
df_all = pd.concat([df1,df2,df3,df4, df5,df6,df7,df8, df9,df10,df11, df12, df13,df14,df15, df16, df17])
df_all

,Unnamed: 0,item_id,product,brand,total_reviews,user_name,review_title,review_text,review_rating,helpful_votes,not_helpful_votes,verified_purchase,sephora_employee,incentivized
0,0,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",flDeb,Too thick and sticky.,More like a sticky paste than cream. I also do...,1.0,0.0,1.0,1,0,0
1,1,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",KitKatChi2018,Heaven for glowy normal to combo skin,I was NOT expecting to love this moisturizer a...,4.0,2.0,0.0,0,0,0
2,2,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",Scorpiobabe93,Nice but ridiculously overhyped.,It’s moisturizing and definitely feels very lu...,3.0,13.0,5.0,0,0,0
3,3,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",purrrsuasive1,NaN,I have been using this for years 1 ounce last ...,5.0,12.0,13.0,0,0,0
4,4,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",EmilijaLVS,Not worth the hype,I really don’t get the hype. I got a mini size...,2.0,26.0,8.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47223,355,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,CrazyxoLizzy,VERY HYDRATING,My face was screaming for hydration and the Th...,4.0,0.0,0.0,0,0,0
47224,356,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,JBracamontes,just WOWWWW,I have been using this In the mornings and nig...,5.0,0.0,0.0,0,0,1
47225,357,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,Jayaaah,AUH-MAY-ZINGGGG,"I have combination skin, an oily t zone and su...",5.0,0.0,0.0,0,0,0
47226,358,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,saarah14,All Aboard the Hydration Train!,I have oily skin and I was surprised how amazi...,5.0,0.0,0.0,0,0,1


In [29]:
df = df_all.drop('Unnamed: 0', axis = 1).drop_duplicates()
df

,item_id,product,brand,total_reviews,user_name,review_title,review_text,review_rating,helpful_votes,not_helpful_votes,verified_purchase,sephora_employee,incentivized
0,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",flDeb,Too thick and sticky.,More like a sticky paste than cream. I also do...,1.0,0.0,1.0,1,0,0
1,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",KitKatChi2018,Heaven for glowy normal to combo skin,I was NOT expecting to love this moisturizer a...,4.0,2.0,0.0,0,0,0
2,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",Scorpiobabe93,Nice but ridiculously overhyped.,It’s moisturizing and definitely feels very lu...,3.0,13.0,5.0,0,0,0
3,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",purrrsuasive1,NaN,I have been using this for years 1 ounce last ...,5.0,12.0,13.0,0,0,0
4,1932193,Crème de la Mer Moisturizer,La Mer,"1,326",EmilijaLVS,Not worth the hype,I really don’t get the hype. I got a mini size...,2.0,26.0,8.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47223,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,CrazyxoLizzy,VERY HYDRATING,My face was screaming for hydration and the Th...,4.0,0.0,0.0,0,0,0
47224,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,JBracamontes,just WOWWWW,I have been using this In the mornings and nig...,5.0,0.0,0.0,0,0,1
47225,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,Jayaaah,AUH-MAY-ZINGGGG,"I have combination skin, an oily t zone and su...",5.0,0.0,0.0,0,0,0
47226,2404853,Thirst Trap Juice Hyaluronic Acid & Peptide Hy...,Wishful,422,saarah14,All Aboard the Hydration Train!,I have oily skin and I was surprised how amazi...,5.0,0.0,0.0,0,0,1


In [31]:
df.to_csv('sephora_review_data.csv', index = False)

In [17]:
df['incentivized'].value_counts()

0    118189
1     66449
Name: incentivized, dtype: int64

In [18]:
df['verified_purchase'].value_counts()

0    167293
1     17345
Name: verified_purchase, dtype: int64

In [19]:
66449 + 17345

83794